<a href="https://colab.research.google.com/github/amirsakib16/Introduction-to-Natural-Language-Processing/blob/main/22299222_Amir_Sakib_Saad_CSE440_04_Lab_Assignment02_Fall2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Download the IMDB movie review dataset. Preprocess the review texts by tokenizing, converting to lowercase, and removing punctuation. Then, use CountVectorizer to convert the preprocessed texts into Bag-of-Words feature vectors. Report the dimensions of the resulting feature matrix.**


# **Download the IMDB movie review dataset.**

In [ ]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/dataset/IMDB Dataset.csv")
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


# **Preprocess the review texts by tokenizing**

In [ ]:
!pip install nltk

# **Import Libraries**

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
from nltk.tokenize import word_tokenize
import string

# **Text Pre-processing Operations**

In [ ]:
def lowercase(data):
    data = data.lower()
    result = removeStopWords(data)
    return result
def removePunc(data):
    data = data.translate(str.maketrans("", "", string.punctuation))
    return data
def removeStopWords(data):
    stopword = stopwords.words('english')
    data = removePunc(data)
    token = word_tokenize(data)
    result = []
    for i in token:
        if i not in stopword:
            result.append(i)
    return result
df['clean'] = df['review'].apply(lowercase)

# **CheckPoint**

In [ ]:
def tokenTxt(lst):
    toString = ''
    for i in lst:
        toString+=f"{i} "
    return toString
df['clean'] = df['clean'].apply(tokenTxt)
print(df['clean'].head())

0    one reviewers mentioned watching 1 oz episode ...
1    wonderful little production br br filming tech...
2    thought wonderful way spend time hot summer we...
3    basically theres family little boy jake thinks...
4    petter matteis love time money visually stunni...
Name: clean, dtype: object


# **Import Libraries**

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
corpus = df['clean']
cv = CountVectorizer()

# **Applying Bag of Words**

In [ ]:
def Bag_of_Words(corpus,cv):
    bag_of_words = cv.fit_transform(corpus)
    return bag_of_words
Bag_of_Words(corpus,cv)

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 5028811 stored elements and shape (50000, 180347)>

# **Dimension of the matrix**

In [ ]:
dimension = Bag_of_Words(corpus,cv).shape
print(dimension)

(50000, 180347)


# **Drop the previous raw column**

In [ ]:
df = df.drop('review', axis=1)

In [ ]:
df.head()

,sentiment,clean
0,positive,one reviewers mentioned watching 1 oz episode ...
1,positive,wonderful little production br br filming tech...
2,positive,thought wonderful way spend time hot summer we...
3,negative,basically theres family little boy jake thinks...
4,positive,petter matteis love time money visually stunni...


**Using the same preprocessed IMDB dataset, apply TfidfVectorizer to generate TF-IDF embeddings. Identify the top 10 words with the highest TF-IDF scores.**


# **TF-IDF Implementation**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
matrix = tfidf.fit_transform(df['clean'])

In [ ]:
matrix

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 5028811 stored elements and shape (50000, 180347)>

In [ ]:
feature_names = tfidf.get_feature_names_out()
idf_scores = tfidf.idf_

In [ ]:
feature_names

array(['00', '000', '0000000000001', ..., 'þór', 'יגאל', 'כרמון'],
      dtype=object)

In [ ]:
idf_scores

array([ 9.87388814, 10.721186  , 11.1266511 , ..., 11.1266511 ,
       11.1266511 , 11.1266511 ])

# **Filter out the numbers if any exist and fetch top 10 words**

In [ ]:
idf_series = pd.Series(idf_scores, index=feature_names)
is_purely_numeric = idf_series.index.str.isdigit()
idf_series_filtered = idf_series[~is_purely_numeric]
idf_series_filtered.sort_values(ascending=False).head(10)

,0
כרמון,11.126651
007br,11.126651
יגאל,11.126651
00agent,11.126651
úbercrap,11.126651
østbye,11.126651
özdemir,11.126651
örnek,11.126651
önsjön,11.126651
ïts,11.126651


**Obtain the GloVe embeddings (glove.6B.100d.txt). Perform analogy tasks such as “Teacher - Educate + Heal” and check whether the resulting vector is closest to “Doctor” using the GloVe embeddings.**


# **GloVe initialization**

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
file = "/content/glove.6B.100d.txt"

def Main(glove_file):
    word_vectors = {}
    with open(glove_file, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.array(values[1:], dtype='float32')
            word_vectors[word] = vector
    return word_vectors


glove_model = Main(file)

# **Vectors**

In [ ]:
teacherVec = glove_model['teacher']
educateVec = glove_model["educate"]
healVec = glove_model["heal"]
result = (teacherVec-educateVec)+healVec
doctorVec = glove_model['doctor']

# **Cosine Similarity to find the contextual meaning**

In [ ]:
result_2d = result.reshape(1, -1)
doctorVec_2d = doctorVec.reshape(1, -1)

isDoctor = cosine_similarity(result_2d, doctorVec_2d)
print(isDoctor[0][0])

0.54193926


**Select Brown corpus and load the text data. Preprocess the text by tokenizing, converting to lowercase. Train Word2Vec models using both Skipgram and CBOW on your chosen corpus  (gensim.models.Word2Vec). Evaluate the trained models on word similarity tasks and the same analogy tasks from the previous questions.**


# **Libraries Import**

In [ ]:
!pip install nltk
import nltk
nltk.download('brown')
from nltk.corpus import brown

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [ ]:
brown.categories()

['adventure',
 'belles_lettres',
 'editorial',
 'fiction',
 'government',
 'hobbies',
 'humor',
 'learned',
 'lore',
 'mystery',
 'news',
 'religion',
 'reviews',
 'romance',
 'science_fiction']

In [ ]:
nltk.corpus.brown.raw()
data = nltk.corpus.brown.raw(categories='mystery')

# **Convert the text into lower case**

In [ ]:
data = data.lower()

## **Create Tokens**

In [ ]:
import re
result = re.split(r'([,.:;?_!"()\']|--|\s)', data)
result_final = [i for i in result if i not in (" ", "", "\n")]

In [ ]:
secondaryToken = []
for i in result_final:
    if '/' in i:
        secondaryToken.append(i.split('/')[0])
    else:
        secondaryToken.append(i)
print(secondaryToken)

['\t', 'there', 'were', 'thirty-eight', 'patients', 'on', 'the', 'bus', 'the', 'morning', 'i', 'left', 'for', 'hanover', ',', '', ',', 'most', 'of', 'them', 'disturbed', 'and', 'hallucinating', '.', '', '.', 'an', 'interne', ',', '', ',', 'a', 'nurse', 'and', 'two', 'attendants', 'were', 'in', 'charge', 'of', 'us', '.', '', '.', '\t', 'i', 'felt', 'lonely', 'and', 'depressed', 'as', 'i', 'stared', 'out', 'the', 'bus', 'window', 'at', 'chicago', "'", 's', 'grim', ',', '', ',', 'dirty', 'west', 'side', '.', '', '.', 'it', 'seemed', 'incredible', ',', '', ',', 'as', 'i', 'listened', 'to', 'the', 'monotonous', 'drone', 'of', 'voices', 'and', 'smelled', 'the', 'fetid', 'odors', 'coming', 'from', 'the', 'patients', ',', '', ',', 'that', 'technically', 'i', 'was', 'a', 'ward', 'of', 'the', 'state', 'of', 'illinois', ',', '', ',', 'going', 'to', 'a', 'hospital', 'for', 'the', 'mentally', 'ill', '.', '', '.', '\t', 'i', 'suddenly', 'thought', 'of', 'mary', 'jane', 'brennan', ',', '', ',', 'the'

In [ ]:
secondaryToken = [secondaryToken]

## **Word to Vector Training (Word2Vec)**

In [ ]:
!pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 14.8 MB/s eta 0:00:00


# **Library Installation**

In [ ]:
import gensim
from gensim.models import Word2Vec

# **Parameter Initialization**

In [ ]:
model_SkipGram = Word2Vec(
    vector_size = 10,
    window = 3,
    sg = 1,
    min_count = 1
)

model_CBOW = Word2Vec(
    vector_size = 10,
    window = 3,
    sg = 0,
    min_count = 1
)

In [ ]:
vocSG = model_SkipGram.build_vocab(secondaryToken)
vocCBOW = model_CBOW.build_vocab(secondaryToken)

In [ ]:
model_SkipGram.train(
    secondaryToken,
    total_examples = len(secondaryToken),
    epochs = 10
)

model_CBOW.train(
    secondaryToken,
    total_examples = len(secondaryToken),
    epochs = 10
)

(100000, 789620)

In [ ]:
window_VectorSG = model_SkipGram.wv['window']
print(window_VectorSG)
window_VectorCBOW = model_CBOW.wv['window']
print(window_VectorCBOW)
print("Are they Similar?")
print(window_VectorCBOW==window_VectorSG)

[ 0.13861766  0.03729064  0.79253393 -0.08119792  0.40011376 -0.03215734
  0.80496037  0.7423255  -0.633314   -0.3340784 ]
[ 0.10131656  0.01082024  0.48878738 -0.02368038  0.2526813  -0.16320974
  0.6818564   0.66243577 -0.4615717  -0.2679743 ]
Are they Similar?
[False False False False False False False False False False]


In [ ]:
morning_VectorSG = model_SkipGram.wv['morning']
print(morning_VectorSG)
morning_VectorCBOW = model_CBOW.wv['morning']
print(morning_VectorCBOW)
print("Are they Similar?")
print(morning_VectorCBOW==morning_VectorSG)

[ 0.19713293  0.04123488  0.47083336 -0.14844988  0.17412524  0.16972551
  0.34280857  0.6500122  -0.45479947 -0.18095335]
[ 0.07593213  0.01658932  0.2213998   0.03251586  0.07568075  0.03472733
  0.20051526  0.2981543  -0.18605517 -0.0764081 ]
Are they Similar?
[False False False False False False False False False False]


In [ ]:
morning_window = cosine_similarity(morning_VectorSG.reshape(1, -1), window_VectorSG.reshape(1, -1))
print(morning_window[0][0])

0.9407993


In [ ]:
morning_window = cosine_similarity(morning_VectorCBOW.reshape(1, -1), window_VectorCBOW.reshape(1, -1))
print(morning_window[0][0])

0.9547261
